# Test the self hosted service 
The service is set up with docker. To find out how to change the possibilities what to control the possible routing parameters from the outside in this notbook a call for the service is implemented. 

In [1]:
import pandas as pd
from routingpy import ORS, Google
from routingpy.exceptions import RouterApiError
import folium
import requests
import json

In [13]:
class RoutingTool: 
    """Calculate different routes for each given start and end point."""
    def __init__(self):
        '''Define url.'''
        self.base_url_ors = 'http://localhost:8080/ors/v2/directions'
        self.setting_ors_cargo_bike()
        
    def setting_ors_cargo_bike(self, elevation = True, avoid_features = ["junction", "steps"], steepness_difficulty = 0):

        ors_cargo_bike_green_args = {
            'elevation': elevation, # take elevation into account
            'language': 'de',
            'options': {
                "avoid_features": avoid_features,# avoid steps
                'profile_params': {
                    'weightings': {
                        'steepness_difficulty': steepness_difficulty, # Beginner steepness difficulty
                        'green': 1
                    },
                },
            },
            'extra_info': [
                'waytype',
                'surface'
            ],
        }     
        ors_cargo_bike_args = {
            'elevation': elevation, # take elevation into account
            'language': 'de',
            'options': {
                'avoid_features': avoid_features,# avoid steps
                'profile_params': {
                    'weightings': {
                        'steepness_difficulty': steepness_difficulty, # Beginner steepness difficulty
                    },
                },
            },
            'extra_info': [
                'waytype',
                'surface'
            ],
        }   
        self.args = [ors_cargo_bike_green_args, ors_cargo_bike_args]

    def calculate_directions(self, coordinates):
        routes = []
        client, profile, args, name = ORS(base_url = self.base_url_ors), 'cycling-cargo', self.args, 'ORS'
        try:
            for arg in args:
                print("Trying directions: ", name)
                route = client.directions(locations=coordinates, profile=profile, **arg)
                print(route)
                routes.append((route, name))
        except RouterApiError as rae:
            # print("Error", rae.message['error'])
            # self.rae = rae
            # return False
            print(rae)
        except KeyError as ke:
            # print("Error", ke.message['error'])
            # return False
            print(ke)
        self.routes = routes
        return True
    def direction_wgs84(self):
        '''Return the coordinates in wgs84.'''
        return self.routes
    def direction_etrs89(self):
        '''Return the coordinates in etrs89.'''
        try:
            ors_coords = [self.convert_coord_systems(geom) for geom in self.routes[0][0].geometry]
        except:
            return False
        etrs89  = ors_coords
        return etrs89
        


In [14]:
routes = []
coord = [[8.67710307065867, 49.4229726228998], [8.683560278482277, 49.39729589125126]] 
tool_track = RoutingTool()
if tool_track.calculate_directions(coord):
    print(tool_track.routes)
    routes.append(tool_track.routes[0][0])
    routes.append(tool_track.routes[1][0])

Trying directions:  ORS
'content-type'
[]


# Alternative approach to calling local ORS

In [2]:
def swap_lat_lon(coordinates):
        return list(map(lambda point: [point[1], point[0]], coordinates))

In [11]:
coords_1 = [[52.77782, 13.24303], [52.5847, 13.4081], [52.5562, 13.3845], [52.4039, 13.4043]]
coords_2 = [[52.77782, 13.24303], [52.5563, 13.441]]
coords_3 = [[49.429321002133264, 8.645513302164934], [49.382699440280376, 8.672026684572918]]
coords_4 = [[49.41420893897647, 8.71626905022367], [49.421054562823805, 8.686822964497205], [49.43177943919527, 8.705671632343325]]
coords_5 = [[49.407022003174916, 8.690847910825031], [49.404651744895276, 8.677084284967348]]
coords_6 = [[49.40003, 8.69061], [49.39666, 8.68972]]

coords_7 = [[47.37814421396306, 8.541416168049837], [47.36949803239042, 8.539957046453003]]


coords_1 = swap_lat_lon(coords_1)
coords_2 = swap_lat_lon(coords_2)
coords_3 = swap_lat_lon(coords_3)
coords_4 = swap_lat_lon(coords_4)
coords_5 = swap_lat_lon(coords_5)
coords_6 = swap_lat_lon(coords_6)
coords_7 = swap_lat_lon(coords_7)


check_url = 'http://localhost:8080/ors/v2/health'

check_response = requests.get(check_url)
ideal_response = {"status":"ready"}

if check_response.json() == ideal_response:

    url = 'http://localhost:8080/ors/v2/directions/cycling-cargo'

    headers = {
    'Content-Type': 'application/json; charset=utf-8',
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8'
    }   

    data_1 = {
        "coordinates": coords_7,
        # 'options': {
        #     'profile_params': {
        #         'weightings': {
        #             'green': 0.8
        #         },
        #     },
        # },
        # "extra_info": ["green"],
        "instructions":"false",
        "preference":"recommended",
        "units":"m"
    }

    data_2 = {
    "coordinates": coords_7,
    # 'options': {
    #     'profile_params': {
    #         'weightings': {
    #             'green': 0.0
    #         },
    #     },
    # },
    # "extra_info": ["green"],
    "instructions":"false",
    "preference":"recommended",
    "units":"m"
    }

    response_1 = requests.post(url, headers=headers, data=json.dumps(data_1))

    response_2 = requests.post(url, headers=headers, data=json.dumps(data_2))

    # Remove comments below to save file

    filename = 'response-custom.json'

    with open(filename, 'w') as f:
        json.dump(response_1.json(), f, indent=4)

else:
    print("ORS not ready...")
    print("Response: ", check_response.json())

In [12]:
response_1 = response_1.json()
response_2 = response_2.json()

In [13]:
coords_7

[[8.541416168049837, 47.37814421396306],
 [8.539957046453003, 47.36949803239042]]

In [14]:
response_1

{'error': {'code': 2010,
  'message': 'Could not find routable point within a radius of 400.0 meters of specified coordinate 0: 8.5414162 47.3781442.'},
 'info': {'engine': {'build_date': '2023-10-06T09:54:31Z', 'version': '8.0'},
  'timestamp': 1696852485004}}

In [8]:
response_2

{'bbox': [8.689218, 49.396658, 8.690602, 49.400031],
 'routes': [{'summary': {'distance': 427.9, 'duration': 70.0},
   'bbox': [8.689218, 49.396658, 8.690602, 49.400031],
   'geometry': 'emolHgk`t@^Jv@TVLVVNZHTEHARBNFDLAJJXd@Xf@JJH@FA\\ADAZA@?@?TCD?JA~AYz@MZOd@IJAx@K',
   'way_points': [0, 31],
   'legs': []}],
 'metadata': {'attribution': 'openrouteservice.org, OpenStreetMap contributors, tmc - BASt',
  'service': 'routing',
  'timestamp': 1696586237106,
  'query': {'coordinates': [[8.69061, 49.40003], [8.68972, 49.39666]],
   'profile': 'cycling-cargo',
   'preference': 'recommended',
   'format': 'json',
   'units': 'm'},
  'engine': {'version': '8.0',
   'build_date': '2023-10-06T09:54:31Z',
   'graph_date': '2023-10-06T09:55:56Z'}}}

In [25]:
class DisplayRoute():
    def swap_lat_lon(self,coordinates):
        return list(map(lambda point: [point[1], point[0]], coordinates))
    def display_map(self,routes):
        results = self.setting_up_api(routes)
        my_map = folium.Map()

        for route, name, color in results:
            points = self.swap_lat_lon(route.geometry) 
            feature_group = folium.FeatureGroup(name=name)
            folium.PolyLine(points, color=color, opacity=0.5, tooltip=name).add_to(feature_group)
            feature_group.add_to(my_map)

        folium.LayerControl(collapsed=False, hideSingleBase=True).add_to(my_map)
        my_map.fit_bounds(my_map.get_bounds())
        return my_map
    def setting_up_api(self, routes):
        settings = []
        settings.append(routes)
        settings.append((routes[0], 'Green', 'green'))
        settings.append((routes[1], ' Not Green', 'blue'))
        return settings

In [ ]:
map_routes = DisplayRoute()
map_routes.display_map(response)

# Alternative way to display routes

## NON-GREEN ROUTE

In [9]:
import folium
import polyline

# Decode polyline
geometry_1 = response_1['routes'][0]['geometry']
geometry_2 = response_2['routes'][0]['geometry']

decoded_1 = polyline.decode(geometry_1)
decoded_2 = polyline.decode(geometry_2)

# Create a map centered at the average latitude and longitude of your route
map_center = [sum(lat for lat, _ in decoded_1) / len(decoded_1), sum(lon for _, lon in decoded_1) / len(decoded_1)]
route_map = folium.Map(location=map_center, zoom_start=14)

# Create feature groups
route_1 = folium.FeatureGroup(name='Route 1')
route_2 = folium.FeatureGroup(name='Route 2')

# Add the first decoded polyline to the route_1 feature group
folium.PolyLine(locations=decoded_1, color='blue', weight=2.5, opacity=1).add_to(route_1)
route_1.add_to(route_map)  # Add feature group to map

# Add the second decoded polyline to the route_2 feature group
folium.PolyLine(locations=decoded_2, color='red', weight=2.5, opacity=1).add_to(route_2)
route_2.add_to(route_map)  # Add feature group to map

# Add layer control to toggle routes on map
folium.LayerControl().add_to(route_map)

# Display the map
route_map


# Green Route

In [40]:
import folium
import polyline

# Decode polyline
geometry_1 = response_1['routes'][0]['geometry']
geometry_2 = response_2['routes'][0]['geometry']

decoded_1 = polyline.decode(geometry_1)
decoded_2 = polyline.decode(geometry_2)

# Create a map centered at the average latitude and longitude of your route
map_center = [sum(lat for lat, _ in decoded_1) / len(decoded_1), sum(lon for _, lon in decoded_1) / len(decoded_1)]
route_map = folium.Map(location=map_center, zoom_start=14)

# Create feature groups
route_1 = folium.FeatureGroup(name='Route 1')
route_2 = folium.FeatureGroup(name='Route 2')

# Add the first decoded polyline to the route_1 feature group
folium.PolyLine(locations=decoded_1, color='blue', weight=2.5, opacity=1).add_to(route_1)
route_1.add_to(route_map)  # Add feature group to map

# Add the second decoded polyline to the route_2 feature group
folium.PolyLine(locations=decoded_2, color='red', weight=2.5, opacity=1).add_to(route_2)
route_2.add_to(route_map)  # Add feature group to map

# Add layer control to toggle routes on map
folium.LayerControl().add_to(route_map)

# Display the map
route_map
